# Let's get a tidy dataset ouf of a questionnary's file
Difficulty

    * Multiple occurences per user
    * One row per response
    * Multiple answers questions


In [1]:
import pandas as pd

df = pd.read_excel("src/modele-export.xlsx", header=None)

# futurs columns names
hor_cols = ["Client", "Campagne", "Enquête", "Date de début", "Date de fin", "user"]

# values that are going to be columns names and corresponding values
vert_cols = ["questions", "qtype", "response"]

# set multiple answers questions
multiples = ["Question 6"]

# columns naming
df.columns = hor_cols + vert_cols
df.head(2)

,Client,Campagne,Enquête,Date de début,Date de fin,user,questions,qtype,response
0,Github,CampagneA,EnquêteA,Date de début1,Date de fin2,Utilisateur 1,Question 1,"Mood (0, very sad to 5, very happy)",Reponse Utilisateur 1 9
1,Github,CampagneA,EnquêteA,Date de début1,Date de fin2,Utilisateur 1,Question 2,Single-closed choice,Reponse Utilisateur 1 2


In [2]:
# creating destination dataframe with all duplicates
new_df =  df[hor_cols].drop_duplicates().set_index('user')

# columns indicating if each Multi-closed have been checked or not
for multiple in multiples :
    new_df2 = pd.get_dummies(df[df.questions == multiple][["response", "user"]], columns=['response'])
    new_df2 = new_df2.groupby('user').sum()    
    ## renaming columns with the question name first
    new_df2.rename(columns=lambda x: multiple + " : " + x, inplace=True)
    new_df = pd.concat([new_df, new_df2], axis=1)

new_df.head(2)

,Client,Campagne,Enquête,Date de début,Date de fin,Question 6 : response_6#1,Question 6 : response_6#2,Question 6 : response_6#3
user,,,,,,,,
Utilisateur 1,Github,CampagneA,EnquêteA,Date de début1,Date de fin2,1,1,0
Utilisateur 2,Github,CampagneA,EnquêteA,Date de début1,Date de fin2,0,1,0


In [3]:
new_df = new_df.reset_index()
df = df.set_index('user')
new_df

,user,Client,Campagne,Enquête,Date de début,Date de fin,Question 6 : response_6#1,Question 6 : response_6#2,Question 6 : response_6#3
0,Utilisateur 1,Github,CampagneA,EnquêteA,Date de début1,Date de fin2,1,1,0
1,Utilisateur 2,Github,CampagneA,EnquêteA,Date de début1,Date de fin2,0,1,0
2,Utilisateur 3,Github,CampagneB,EnquêteA,Date de début1,Date de fin2,1,1,1


In [4]:
# listing the questions with one answer
question_cols = ['Question 1', 'Question 2', 'Question 3', 'Question 4', 'Question 5', 'Question 7']

# getting the corresponding answer
def get_question(var,col) :
    return(df[df["questions"] == col].loc[str(var),"response"])
    
for col in question_cols :
    new_df[col] = new_df.user.apply(get_question, col=col)
    
new_df.head(2)

,user,Client,Campagne,Enquête,Date de début,Date de fin,Question 6 : response_6#1,Question 6 : response_6#2,Question 6 : response_6#3,Question 1,Question 2,Question 3,Question 4,Question 5,Question 7
0,Utilisateur 1,Github,CampagneA,EnquêteA,Date de début1,Date de fin2,1,1,0,Reponse Utilisateur 1 9,Reponse Utilisateur 1 2,Reponse Utilisateur 1 3,Reponse Utilisateur 1 4,Reponse Utilisateur 1 5,Reponse Utilisateur 1 8
1,Utilisateur 2,Github,CampagneA,EnquêteA,Date de début1,Date de fin2,0,1,0,Reponse Utilisateur 2 7,Reponse Utilisateur 2 2,Reponse Utilisateur 2 3,Reponse Utilisateur 2 4,Reponse Utilisateur 2 5,Reponse Utilisateur 2 7


In [5]:
## saving
new_df = new_df.set_index('user')
new_df.to_csv("final.csv")